# 7章．畳み込みニューラルネットワーク

畳み込みニューラルネットワーク(convolutional neural network: CNN)は，画像認識や音声認識など，至るところで使われている．特に画像認識のコンペティションでは，ディープラーニングによる手法のほとんどすべてがCNNをベースとしている．

## 7.1 全体の構造

全結合層：これまで見てきたニューラルネットワークのような，隣接する層のすべてのニューロン間で結合があるもの．これまではAffineレイヤで実装した．

#### ポイント（一般的なCNNでよくみられる構成）

CNNのレイヤのつながり順：Convolution - ReLU - (Pooling)

出力層に近い層では，これまでの「Affine-ReLU」という組み合わせが用いられる．

最後の出力層においては「Affine-Softmax」の組み合わせが用いられる．

## 7.2 畳み込み層

### 7.2.1 全結合層の問題点

#### 問題点
画像の場合は3次元のデータを平らな1次元データに変換する＝データの形状が無視される．

##### 本質を見逃してしまう例
- 空間的に近いピクセルは似たような値であるケース
- RGBの各チャネル間にはそれぞれに密接な関連性があるケース
- 距離の離れたピクセルどうしはあまり関わりがないケース

#### 畳み込み層(Convolutionレイヤ)は，形状を維持する
- 特徴マップ：畳み込み層の入出力データ
- 入力特徴マップ：畳み込み層の入力データ
- 出力特徴マップ：畳み込み層の出力データ

### 7.2.2 畳み込み演算

畳み込み層では畳み込み演算を行う．

畳み込み演算は，入力データに対して，フィルター（カーネル）を適用する．入力データは縦・横方向形状を持つデータで，フィルターも同様に，縦・横方向の次元を持つ．

畳み込み演算は，入力データに対して，フィルターのウィンドウを一定の間隔でスライドさせ，積和演算の結果を出力の対応する場所に格納していく．

CNNではフィルターの「パラメータ」が全結合層のニューラルネットワークにおける「重み」に対応する．

バイアスは一つだけ存在する．(1 * 1) この一つの値はフィルター適用後のすべての要素に対して加算される．

### 7.2.3 パディング

パディング：畳み込み層の処理を行う前に，入力データの周囲に固定のデータ（例えば0など）で埋めることがある．

パディングを行う理由：出力サイズを調整すること．畳み込み演算を何度も繰り返すことで出力サイズが1になり，それ以上の畳み込み演算を適用できなくなる事態を避ける．空間的なサイズを一定に保ち，次の層へデータを渡すためにパディングは利用される．

### 7.2.4 ストライド

ストライド：フィルターを適用する位置の間隔．ここまではすべてストライドが1．ストライドを大きくすると出力サイズは小さくなる．

##### 以下の式で，入力サイズ，パディング，ストライド，フィルターサイズを用いて出力サイズを計算することができる．


$$
    OH = \frac{H + 2P - FH}{S}+1
$$

$$
   OW = \frac{W + 2P - FW}{S}+1
$$

- OH: 出力高さ
- OW: 出力幅
- H: 入力高さ
- W: 入力幅
- P: 幅
- S: ストライド
- FH: フィルター高さ
- FW: フィルター幅

##### 注意点
式の作成時には分数の部分が割り切れるようにそれぞれの値を設定する必要がある．出力サイズが割り切れない場合（結果が小数の場合）は，エラーを出力するなどして対応する必要があります．ディープラーニングのフレームワークによっては，値が割り切れないときは最も近い整数に丸めるなどして，特にエラーを出さないで先に進むような実装をする場合もあります．

### 7.2.5 3次元データの畳み込み演算

画像の場合，縦，横方向に加えてチャンネル方向も合わせた3次元データを扱う必要がある．
##### 注意点
入力データとフィルターのチャンネル数は同じ値にするということ．

### 7.2.6 ブロックで考える

3次元の畳み込み演算は，データやフィルターを直方体のブロックで考えるとわかりやすい．

- ブロック：3次元の直方体

畳み込み演算の出力をチャンネル方向にも複数持たせるには複数のフィルターが必要になる．

フィルターの重みデータは4次元のデータとして，
- (output_channel, input_channel, height, width)

畳み込み演算では（全結合層と同じく）バイアスが存在する．バイアスは1チャンネルごとに一つだけデータを持つ．フィルターの出力結果に対してチャンネルごとに，同じ値が加算される．

### 7.2.7 バッチ処理

畳み込み演算にも同じようにバッチ処理に対応する．各層を流れるデータは4次元データとして格納する．
- (batch_num, channel, height, width)
ネットワークには4次元のデータが流れるが，これはN個のデータに対して畳み込み演算が行われていることを意味する．つまりN回分の処理を一回にまとめて行っている．

## 7.3 プーリング層

##### プーリング：縦・横方向の空間を小さくする演算
2×2Maxプーリングをストライド2で行った場合の処理

→2×2を対象領域として最大値をとる．

一般的にプーリングのウィンドウサイズと，ストライドは同じ値に設定する．

プーリングにはMaxプーリングの他に，Averageプーリングなどがある．画像認識の分野においては，主にMaxプーリングが使われる．

### 7.3.1 プーリング層の特徴

- 学習するパラメータがない
：対象領域から最大値や平均値をとるだけの処理なので学習すべきパラメータがない．
- チャンネル数は変化しない
：チャンネル数は変化せずチャンネルごとに独立して計算が行われる．
- 微小な位置変化に対してロバスト（頑健）
：入力データの小さなズレに対して，プーリングは同じような結果を返す．そのため入力データの微小なズレに対してロバストである．

## 7.4 Convolution / Poolingレイヤの実装

### 7.4.1 4次元配列

In [8]:
# -*- coding:utf-8 -*-

import numpy as np
x = np.random.rand(10, 1, 28, 28)

# 形状の確認
x.shape

# 一つ目のデータの1チャンネル目の空間データにアクセスする
x[0, 0].shape

(28, 28)

### 7.4.2 im2colによる展開

im2col：フィルター（重み）にとって都合の良いように入力データを展開する関数．

im2colによって，展開すると展開後の要素数は元のブロックの要素数よりも多くなる．そのため通常よりも多くのメモリを消費するが，大きな行列をまとめて計算することはコンピュータにとって都合がいい．（行列計算のライブラリなどは行列の計算実装が高度に最適化されているため．）行列の計算に帰着させることは有効となる．

### 7.4.3 Convolutionレイヤの実装

im2colのインタフェース：im2col(input_data, filter_h, filter_w, stride, pad)
- input_data：（データ数，チャンネル，高さ，横幅）の4次元配列からなる入力データ
- filter_h：フィルターの高さ
- filter_w：フィルターの横幅
- stride：ストライド
- pad：パディング

In [14]:
# im2colを利用して入力データを2次元配列に展開

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

# バッチサイズ1
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

# バッチサイズ10
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [15]:
# 畳み込み層をim2colを使って実装する
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        """ フィルター（重み）とバイアス，ストライドとパディングを
        引数として受け取る"""
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w = self.W.reshape(FN, -1)  # フィルターの展開
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 7.4.4 Poolingレイヤの実装

In [17]:
# Poolingレイヤのforward処理の実装
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 最大値
        out = np.max(col, axis=1)
        # 整形
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    
        return out

## 7.5 CNNの実装

In [19]:
# 手書き数字認識を行うCNNを組み立てる
# ネットワークの構成はConvolution→ReLU→Pooling→Affine→ReLU→Affine→Softmax

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num': 30, 'filter_size': 5,
                           'pad': 0, 'stride': 1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
                            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * \
                              (conv_outpus_size/2))
        
        # 重みの初期化を行うパート
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0],
                                           filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, 
                                           hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(outpus_size)
        
        # 必要なレイヤを生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], 
                                          self.params['b1'],
                                          conv_param['stride'],
                                          conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],
                                       self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],
                                       self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    # 推論を行うpredict, 損失関数の値を求めるloss
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 設定
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads

MNISTデータセットを学習するコードを用いて実装すると認識率の確認ができる．

畳み込み層とプーリング層は画像認識では必須のモジュール．画像という空間的な形状のある特性をCNNはうまく読み取り，手書き数字認識においても高精度の認識を実現することができる．

## 7.6 CNNの可視化

#### CNNで用いられる畳み込み層は何を見ているのか
→畳み込み層の可視化を通じて，CNNで何が行われているのか探索していく．

### 7.6.1 １層目の重みの可視化

### 7.6.2 階層構造による情報抽出

## 7.7 代表的なCNN

### 7.7.1 LeNet

### 7.7.2 AlexNet

## 7.8 まとめ

### 本章で学んだこと
- CNNは，これまでの全結合層のネットワークに対して，畳み込み層とプーリング層が新たに加わる．
- 畳み込み層とプーリング層は，im2col（画像を行列に展開する関数）を用いるとシンプルで効率の良い実装ができる．
- CNNの可視化によって，層が深くなるにつれて高度な情報が抽出されていく様子がわかる．
- CNNの代表的なネットワークには．LeNetとAlexNetがある．
- ディープラーニングの発展に，ビッグデータとGPUが大きく貢献している．